In [1]:
#   IMPORTS
from sktime.utils.plotting import plot_series
from sktime.forecasting.base import ForecastingHorizon
from sktime.forecasting.model_selection import temporal_train_test_split
from sktime.forecasting.naive import NaiveForecaster
from sktime.performance_metrics.forecasting import MeanAbsolutePercentageError
from sktime.performance_metrics.forecasting import mean_absolute_error
from sktime.forecasting.compose import make_reduction
from sktime.forecasting.fbprophet import Prophet
from sktime.forecasting.ets import AutoETS
from sktime.forecasting.exp_smoothing import ExponentialSmoothing
from sktime.forecasting.bats import BATS
from sktime.forecasting.tbats import TBATS
from sktime.forecasting.var import VAR
from sktime.forecasting.arima import AutoARIMA
from sktime.forecasting.arima import ARIMA
from sktime.forecasting.sarimax import SARIMAX
from sktime.forecasting.theta import ThetaForecaster
from sktime.forecasting.trend import TrendForecaster
from sktime.forecasting.compose import EnsembleForecaster

from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from prophet.forecaster import Prophet 

from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import StackingRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import BaseEnsemble
from sklearn.metrics import mean_squared_log_error

from pandas import read_csv
import pandas as pd 
import numpy as np
import warnings
import datetime
import warnings
warnings.filterwarnings("ignore")

In [2]:
#   LOAD IN DATA
data = pd.read_csv('data_for_inequalities.csv')
#View head
data.head()

,Date,SG_Y
0,2017/12,76.500
1,2018/01,82.750
2,2018/02,85.500
3,2018/03,86.125
4,2018/04,88.700


In [3]:
#   USER DEFINED
#Set target_col to column name you want to forecast 
#Example uses column named SG_Y
target_col = 'SG_Y'

In [4]:
#   TRAIN/TEST SPLIT 
y = data[target_col]
y_train, y_test = temporal_train_test_split(y, test_size=12)

In [5]:
#Set Forecasting horizon ------ Forecast next 730 hours (730 hours in a month)
fh = np.arange(1, len(y_test) +1)
fh

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12])

In [6]:
#Create estimator and mae score array to be appended with models in ensemble 
estimators=[]
mae_scores=[]

In [7]:
#   Build Auto ETS Model and score MAE in sample Forecast
model1 = AutoETS(auto=True, sp=4, n_jobs=-1)
#Append estimators array by adding model1
estimators.append(("Auto ETS",model1))
#Fits model on training data
model1.fit(y_train)
#Forecasts
target_pred1 = model1.predict(fh)
#Calculates MAE for model1
ETSmae = mean_absolute_error(y_test, target_pred1, symmetric=False)
#Calculate RMSLE score for ensemble model
ETSrmsle = mean_squared_log_error(y_test, target_pred1, squared=False)
#Append mae_scores array by adding model1 mae score 
mae_scores.append(ETSmae)
#Prints scores for model1
print('AUTO ETS Model - MAE: %.3f' % (ETSmae))
print("AUTO ETS Model - RMSLE: %.3f" % ETSrmsle)
print('--------Auto ETS Forecast--------')
target_pred1

AUTO ETS Model - MAE: 2.601
AUTO ETS Model - RMSLE: 0.029
--------Auto ETS Forecast--------


36    103.960567
37    103.995618
38    104.023659
39    104.046092
40    104.064038
41    104.078394
42    104.089880
43    104.099068
44    104.106419
45    104.112300
46    104.117004
47    104.120768
dtype: float64

In [8]:
#Bagging
#Define the bagging ensemble
BaggingEnsemble = BaggingRegressor()
ensemble_model = make_reduction(BaggingEnsemble)
#Fit model on training data 
ensembleBagging = ensemble_model.fit(y_train)
#Forecast
target_pred = ensembleBagging.predict(fh)
#Calculate MAE score for ensemble model
Ensemblemae = mean_absolute_error(y_test, target_pred)
#Calculate RMSLE score for ensemble model
EnsembleRMSLE = mean_squared_log_error(y_test, target_pred, squared=False)
#Print scores for model3
print("Ensemble MAE: %.3f" % Ensemblemae)
print("Ensemble RMSLE: %.3f" % EnsembleRMSLE)
print(target_pred)

Ensemble MAE: 2.575
Ensemble RMSLE: 0.029
36    104.166667
37    104.183333
38    103.925000
39    104.166667
40    104.125000
41    104.125000
42    104.125000
43    104.125000
44    104.125000
45    104.125000
46    104.125000
47    104.125000
dtype: float64
